### Necessary installations

In [24]:
#pip install tensorflow

In [2]:
#pip install /path/to/requirements.txt

### Necessary imports

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import openpyxl
import os
import sys
import shutil
import time
from tqdm import tqdm

In [4]:
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from keras_facenet import FaceNet

In [5]:
# Check tensorflow version (written in 2.16.1)
print("TensorFlow version:", tf.__version__)

# Check GPU availability
# Unfortunately, tensorflow 2.16 currently does not support GPU for macOS with M2
print("GPU available:", tf.config.list_physical_devices('GPU'))

# Check python version (written in 3.12)
print(f'Python version: {sys.version}')

TensorFlow version: 2.16.1
GPU available: []
Python version: 3.12.0 (v3.12.0:0fb18b02c8, Oct  2 2023, 09:45:56) [Clang 13.0.0 (clang-1300.0.29.30)]


### Import methods from methods.py

In [22]:
from methods import *

##### Set global variables

In [7]:
embedder = FaceNet()

##### Prepare directory

In [8]:
# Method to remove the .DS_Store files that unnoticably appear in the directory
def delete_ds_store_files(directory):
    for ds_store_file in directory.rglob(".DS_Store"):
        try:
            if ds_store_file.is_file():
                ds_store_file.unlink()  # Delete the file
                print(f"Deleted: {ds_store_file}")
            else:
                print(f"Skipping: {ds_store_file} (not a file)")
        except Exception as e:
            print(f"Error deleting {ds_store_file}: {e}")

In [10]:
# Delete .DS_Store files
delete_ds_store_files(Path('./datasets/unique_isolated_pol_faces'))

Deleted: datasets/unique_isolated_pol_faces/.DS_Store


In [11]:
nos_articles_election_period = pd.read_excel('./datasets/nos_articles_election_period.xlsx')

### Filter out sports articles

In [19]:
nos_articles_all_categories = sorted(nos_articles_election_period['category'].unique())


In [23]:
sports_categories

['Alpineskiën',
 'American football',
 'Atletiek',
 'Autosport',
 'BMX',
 'Baanwielrennen',
 'Badminton',
 'Basketbal',
 'Beachvolleybal',
 'Biljarten',
 'Boksen',
 'Cricket',
 'Curling',
 'Dammen',
 'Darts',
 'Futsal',
 'Golf',
 'Handbal',
 'Hockey',
 'IJshockey',
 'Judo',
 'Kanovaren',
 'Korfbal',
 'Kunstschaatsen',
 'Marathonschaatsen',
 'Motocross',
 'Motorsport',
 'Mountainbiken',
 'Paardensport',
 'Paardensport dressuur',
 'Paardensport eventing',
 'Paardensport springen',
 'Paralympische sport',
 'Roeien',
 'Rugby',
 'Schaatsen',
 'Schaken',
 'Shorttrack',
 'Skateboarden',
 'Skeleton',
 'Skispringen',
 'Snooker',
 'Snowboarden',
 'Sport algemeen',
 'Tafeltennis',
 'Tennis',
 'Triatlon',
 'Turnen',
 'Veldrijden',
 'Voetbal',
 'Waterpolo',
 'Zeilen',
 'ZwemmenVoetbal']